In [ ]:
import os
os.environ["DATABRICKS_TOKEN"] = sc.getLocalProperty("spark.databricks.token")

In [ ]:
%sh

cat > ~/.databrickscfg << EOF
[DEFAULT]
host = https://dbc-dfecb9db-32a5.cloud.databricks.com
token = $DATABRICKS_TOKEN
EOF

In [ ]:
%sh

# CLUSTER_SPEC='{ "cluster_name":"", "spark_version":"13.2.x-scala2.12", "instance_pool_id":"0703-011504-era331-pool-7mt36cl2", "driver_instance_pool_id":"0703-011504-era331-pool-7mt36cl2", "data_security_mode": "USER_ISOLATION", "runtime_engine":"STANDARD", "autoscale": { "min_workers":2, "max_workers":8 } }'

CLUSTER_SPEC='{
        "cluster_name":"",
        "spark_version":"13.2.x-scala2.12",
        "spark_conf": {
            "spark.hadoop.fs.s3a.credentialsType": "AssumeRole",
            "spark.sql.legacy.parquet.int96RebaseModeInRead": "LEGACY",
            "spark.hadoop.fs.s3.impl": "com.databricks.s3a.S3AFileSystem",
            "spark.hadoop.fs.s3a.acl.default": "BucketOwnerFullControl",
            "spark.databricks.hive.metastore.glueCatalog.enabled": "true",
            "spark.hadoop.fs.s3n.impl": "com.databricks.s3a.S3AFileSystem",
            "spark.sql.legacy.parquet.datetimeRebaseModeInWrite": "CORRECTED",
            "spark.hadoop.fs.s3a.stsAssumeRole.arn": "arn:aws:iam::160325711770:instance-profile/EMR_EC2_DefaultRole",
            "spark.hadoop.fs.s3a.canned.acl": "BucketOwnerFullControl",
            "spark.speculation": "false",
            "spark.hadoop.hive.metastore.glue.catalogid": "160325711770",
            "spark.sql.parquet.writeLegacyFormat": "true",
            "spark.hadoop.fs.s3a.impl": "com.databricks.s3a.S3AFileSystem",
            "spark.sql.hive.metastore.jars": "maven",
            "spark.sql.hive.metastore.version": "1.2.1",
            "spark.databricks.delta.optimizeWrite.enabled": "true"
        },
        "spark_env_vars": {
            "deltalake_catalogue_id": "160325711770",
            "deltalake_assume_role_arn": "arn:aws:iam::160325711770:instance-profile/EMR_EC2_DefaultRole",
            "environment": "prod",
            "cicd_notebook_path_prefix": "CICD/30_delta_prod/"
        },
        "instance_pool_id":"NONE",
        "driver_instance_pool_id":"NONE",
        "data_security_mode":"NONE",
        "runtime_engine":"STANDARD",
        "autoscale": {
            "min_workers":2,
            "max_workers":4
        }
    }'

mkdir -p ~/app/bronze/db_submit; cp /dbfs/FileStore/scripts/db_submit.zip ~/app/bronze/SparkSQL-Kilaris-DB-Job/db_submit/db_submit.zip; unzip -o ~/app/bronze/SparkSQL-Kilaris-DB-Job/db_submit/db_submit.zip -d ~/app/bronze/SparkSQL-Kilaris-DB-Job/db_submit bin/db-submit --wait-for-completion false --source-zip s3://kilaris-aws-databricks-s3-bucket/packages/pipeline-package.zip --dest-zip /tmp/app/bronze/SparkSQL-Kilaris-DB-Job --cluster-spec "$CLUSTER_SPEC" --conf spark.app.name=SparkSQL-Kilaris-DB-Job --conf spark.yarn.stagingDir=hdfs://$(hostname -f):8020/user/hadoop --conf spark.yarn.appMasterEnv.SPARK_HOME=/usr/lib/spark --conf spark.yarn.submit.waitAppCompletion=true --conf spark.port.maxRetries=1000 --conf spark.yarn.tags=SparkSQL-Kilaris-DB-Job --conf spark.executor.asyncEagerFileSystemInit.paths=s3://kilaris-aws-databricks-s3-bucket2 --conf spark.kryoserializer.buffer.max=1024m --conf spark.default.parallelism=512 --conf spark.sql.adaptive.coalescePartitions.initialPartitionNum=512 --conf spark.dynamicAllocation.maxExecutors=6 --instance-profile-arn arn:aws:iam::160325711770:instance-profile/EMR_EC2_DefaultRole --py-files s3://kilaris-aws-databricks-s3-bucket/packages/pipeline-package.zip --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2,org.apache.spark:spark-avro_2.12:3.3.2,com.fasterxml.jackson.core:jackson-databind:2.12.0,awswrangler /tmp/app/bronze/SparkSQL-Kilaris-DB-Job/main.py  --job demo_app --job-type bronze --job-args I delta_emr_source 9999-12-31-01 travel_details_db route_id
